## EDA and Feature Engineering on the processed dataset

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [53]:
# Read in the data
GEO = "texas"

if isinstance(GEO, str):
    df = pd.read_csv(f"../../data/processed/{GEO}_processed.csv")
elif isinstance(GEO, list):
    dfs = []
    for geo in GEO:
        df = pd.read_csv(f"../../data/processed/{geo}_processed.csv")
        dfs.append(df)
    df = pd.concat(dfs)

df.description = df.description.apply(lambda x: eval(x))
df.comments = df.comments.apply(lambda x: eval(x))

print(df.shape)
df.head()

(978, 8)


,review_id,listing_id,description,comments,sentiment,label,name,amenities
0,83097,5456,"[fabulous, location, walking, convention, cent...","[sylvia, very, nice, informal, relaxed, arrive...",4.333333,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","['TV', 'Wifi', 'Air conditioning', 'Kitchen', ..."
1,133337,5456,"[fabulous, location, walking, convention, cent...","[sylvia, picked, airport, gave, beautiful, eve...",4.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","['TV', 'Wifi', 'Air conditioning', 'Kitchen', ..."
2,150928,5456,"[fabulous, location, walking, convention, cent...","[lovely, time, enjoyed, apartment, clean, spac...",3.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","['TV', 'Wifi', 'Air conditioning', 'Kitchen', ..."
3,2706775,5456,"[fabulous, location, walking, convention, cent...","[sylvia, excellent, host, stayed, touch, made,...",4.500000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","['TV', 'Wifi', 'Air conditioning', 'Kitchen', ..."
4,8602878,5456,"[fabulous, location, walking, convention, cent...","[place, cute, little, self, contained, cottage...",4.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","['TV', 'Wifi', 'Air conditioning', 'Kitchen', ..."


### Distribution of labels

First, let's look at the distribution of labels in the processed dataset.

In [54]:
mbad_pcnt = df[df.label == "mbad"].shape[0] / df.shape[0]
mgood_pcnt = df[df.label == "mgood"].shape[0] / df.shape[0]
mbad_or_mgood_pcnt = df[df.label.isin(["mbad", "mgood"])].shape[0] / df.shape[0]
print(
    f"mbad: {mbad_pcnt:.2%}, mgood: {mgood_pcnt:.2%}, mbad or mgood: {mbad_or_mgood_pcnt:.2%}"
)

bar_vals = df.label.value_counts()

fig = px.bar(
    bar_vals,
    x=bar_vals.index,
    y=bar_vals.values,
    title=f"Distribution of labels (GEO={GEO})",
    labels={"index": "Label", "y": "Count"}
)

fig.show()

mbad: 7.36%, mgood: 6.54%, mbad or mgood: 13.91%


### Ngrams

What are the most common ngrams for the different types of reviews? What are the most common overlaps?

In [55]:
def get_consecutive_ngrams(review, n) -> list:
    """Helper function to get ngrams from a review.

    Args:
        review (str): The review to get ngrams from.
        n (int): The number of ngrams to get.

    Returns:
        list: the list of ngrams, joined by underscores.
    """
    if isinstance(review, str):
        review = review.split("")

    return ["_".join(review[i:i+n]) for i in range(len(review)-n-1)]

def series_to_ngrams(series: pd.Series, N):
    n_grams = series.apply(lambda x: get_consecutive_ngrams(x, N))
    return n_grams.explode()

def get_exclusive_sizes(good: set, bad: set, mgood: set, mbad: set) -> dict:
    """Get the sizes of the exclusive sets"""

    only_good = good - bad - mgood - mbad
    only_bad = bad - good - mgood - mbad
    only_mgood = mgood - good - bad - mbad
    only_mbad = mbad - good - bad - mgood
    
    return {
        "good": len(only_good),
        "bad": len(only_bad),
        "mgood": len(only_mgood),
        "mbad": len(only_mbad)
    }

We can look at the number of unique values to each label for different values of N on a line chart.

In [56]:
import plotly.express as px

intersection_sizes = []
for n in [1, 2, 3, 4, 5, 6]:

    good_ngrams = series_to_ngrams(df[df.label == "good"].comments, n).value_counts()
    bad_ngrams = series_to_ngrams(df[df.label == "bad"].comments, n).value_counts()
    mgood_ngrams = series_to_ngrams(df[df.label == "mgood"].comments, n).value_counts()
    mbad_ngrams = series_to_ngrams(df[df.label == "mbad"].comments, n).value_counts()
    intersection_sizes_n = get_exclusive_sizes(
        set(good_ngrams.index),
        set(bad_ngrams.index),
        set(mgood_ngrams.index),
        set(mbad_ngrams.index)
    )

    intersection_sizes_n["n"] = n

    intersection_sizes.append(intersection_sizes_n)


# prepare data
is_df = pd.DataFrame(intersection_sizes)
is_df = is_df.melt(id_vars="n", var_name="label", value_name="count")
is_df["count"] = is_df["count"].astype(int)
is_df["n"] = is_df["n"].astype(int)

# exclude good 
is_df = is_df[is_df.label != "good"]

# plot a line chart
px.line(is_df, 
    x="n", y="count",
    color="label",
    title=f"Unique # of {GEO} n-grams for each label", 
    labels={"n": "N-gram size", "count": "Unique # of n-grams"},
    markers=True
    )

We can see that the number of unique ngrams increases until about n=3, and then levels off. What are the top 10 3-grams for misleading reviews?

In [57]:
import numpy as np
from collections import Counter

# generate wordcloud for misleading reviews
subset = df[df.label.isin(["mbad", "mgood"])]

# get groups of N consecutive words
N = 3
n_grams_3 = series_to_ngrams(subset.comments, N).to_list()

print("Top 10 most common 3-grams in misleading reviews:")
Counter(n_grams_3).most_common(10)

Top 10 most common 3-grams in misleading reviews:


[('within_walking_distance', 8),
 ('would_not_recommend', 6),
 ('not_recommend_staying', 4),
 ('made_u_feel', 3),
 ('great_host_very', 3),
 ('great_place_stay', 3),
 ('solid_internet_connectivity', 3),
 ('walking_distance_lot', 2),
 ('stay_good_location', 2),
 ('good_location_clean', 2)]

Let's manually remove some of these that don't make sense

Now let's look at a word cloud of the 3-grams in misleading reviews.

**NOTE:** Also broken due to pandas library error. Not sure why it happened all of a sudden.

In [58]:
from wordcloud import WordCloud
from PIL import Image

ngrams_str = " ".join(n_grams_3)

# make house-shaped mask
mask = np.array(Image.open("../../house_shape.png"))
# fill transparent areas with white
mask[mask[:, :, 3] == 0] = [255, 255, 255, 255]

# generate wordcloud
wc = WordCloud(
    background_color="white", 
    max_words=1000, 
    width=mask.shape[0]*5,
    height=mask.shape[1]*5,
    mask = mask,
    contour_width=3,
    contour_color='steelblue',
    colormap="twilight_shifted"
).generate(ngrams_str)

wc

# show wordcloud
fig = px.imshow(wc, title=f"Most common {N}-grams in misleading reviews", width=700, height=700)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

Here is a reusable function to add ngrams features to a dataframe.

In [59]:
def ngrams_features(features, n):
    """Add n-gram features to the features dataframe."""

    # one-hot encode ngrams
    df["ngrams"] = df.comments.apply(lambda x: set(get_consecutive_ngrams(x, 3)))

    # get ngrams that are common in misleading reviews
    misleading = df[df.label.isin(["mbad", "mgood"])]
    ng_set = set(series_to_ngrams(subset.comments, n).to_list())

    # one-hot encode ngrams
    ngram_features = {}
    for ngram in ng_set:
        ngram_features[ngram] = df.ngrams.apply(lambda ngrams: 1 if ngram in ngrams else 0)
    ngram_df = pd.DataFrame(ngram_features)
    features = pd.concat([features, ngram_df], axis=1)

    # Only keep ngram features that appear in at least 3 reviews
    min_num_reviews = 3
    num_reviews_ngram_is_in = features.apply(lambda x: x > 0).sum()
    ngram_features_to_drop= num_reviews_ngram_is_in[num_reviews_ngram_is_in < min_num_reviews].index

    features = features.drop(ngram_features_to_drop, axis=1)

    return features

## Word Embeddings and T-SNE
Another way to look at the data is to use word embeddings. We can use the word2vec model from gensim to create word embeddings for each word in the dataset. We can then take the average of all word vectors in a review to get a vector representation of the review. We can then use T-SNE to reduce the dimensionality of the word embeddings to 3 dimensions so we can plot them.

In [66]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec

def embeddings_features(features, emb_vector_size=50) -> pd.DataFrame:

    print("Creating model...")
    model = Word2Vec(df.comments.to_list(), 
            min_count=1,
            vector_size=emb_vector_size,
            workers=3, 
            window=3, 
            sg=1
        )
    
    # generate embeddings
    print("Generating embeddings...")
    get_review_embeddings = lambda review: np.mean([model.wv[word] for word in review if word in model.wv], axis=0)
    embeddings = np.array([get_review_embeddings(review) for review in df.comments.to_list()])
    
    for i in range(embeddings.shape[1]):
        features[f"embedding_{i}"] = embeddings[:, i]
    
    return features


def tsne_features(features, dims=3):

    from sklearn.manifold import TSNE
    print(f"Applying t-SNE... (dim={dims})")
    tsne = TSNE(n_components=dims, random_state=0)
    embeddings_2d = tsne.fit_transform(features)
    embd_df = pd.DataFrame(embeddings_2d, columns=[f"tsne_{i}" for i in range(dims)])

    return embd_df

Let's apply this to the misleading reviews, and then reduce down to 2 dimensions using T-SNE.

In [67]:
# ignore PerformanceWarning
import warnings
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)

embeddings_df = pd.DataFrame()
embeddings_df["review_id"] = df.review_id
embeddings_df["label"] = df.label
embeddings_df = embeddings_features(embeddings_df)

# apply tsne to reduce to `dims` dimensions
dims = 3
embd_df = tsne_features(embeddings_df.drop(columns=["review_id", "label"], axis=1), dims=dims)
embd_df["review_id"] = embeddings_df.review_id
embd_df["label"] = embeddings_df.label

# plot 3d scatter with plotly
fig = px.scatter_3d(embd_df, x="tsne_0", y="tsne_1", z="tsne_2", color="label", title="t-SNE embeddings of reviews", )
# make size of point smaller and opaque
fig.update_traces(marker=dict(size=2, opacity=0.5))
# make inital zoom bigger
fig.update_layout(scene=dict(camera_eye=dict(x=0.9, y=0.9, z=0.9)))
fig.show()

Creating model...
Generating embeddings...
Applying t-SNE... (dim=3)


It looks like the "mbad" labels are mostly together. The "mgood" are kind of in the  middle, but there is still lots of noise of "good" labels in that region.

# Generate Features

With what we know, let's generate some features. I've provided a function for each type of feature, so we can mix and match.

In [69]:
from collections import Counter

# build features
features = pd.DataFrame()
features = embeddings_features(features)

# apply tsne to reduce to `dims` dimensions
dims = 3
features = tsne_features(features, dims=dims)

# add id and label
features["review_id"] = df.review_id
features["label"] = df.label

print(features.shape)
features.head()

Creating model...
Generating embeddings...
Applying t-SNE... (dim=3)
(978, 5)


,tsne_0,tsne_1,tsne_2,review_id,label
0,-0.639876,-4.597953,-1.690420,83097,good
1,-6.714257,4.310233,-0.528321,133337,good
2,-1.117893,-5.458037,2.873919,150928,good
3,-5.982625,-5.602546,-0.022672,2706775,good
4,1.929237,-3.282773,-1.660596,8602878,good


In [65]:
if isinstance(GEO, list):
    geo_display_name = "_".join(GEO)
else:
    geo_display_name = GEO

features.to_csv(f"../../data/processed/features_{geo_display_name}.csv", index=False)